# Part III (4 points +)

The frozenlake problem above is just too simple: you can beat it even with a random policy search. Go solve something more complicated.

Pick __one of the two tasks__:

* __FrozenLake8x8-v0__ - frozenlake big brother. Achieve score >0.7
* __Taxi-v1__ - essentially a maze where you get score for moving passengers to their destinations. Achieve score >-100)

Your homework assignment is beating that score (see tips below).


### Some tips:
* When solving those envs, please make sure your t_max is large enough to finish game with suboptimal policy. For example, __Taxi-v0 only trains if you let it play for 10k+ ticks/session__. For frozenlake8x8 it's less dire.
* Random policy search is worth trying as a sanity check, but in general you should expect the genetic algorithm (or anything you devised in it's place) to fare much better that random.
* While _it's okay to adapt the tabs above to your chosen env_, make sure you didn't hard-code any constants there (e.g. 16 states or 4 actions).
* `print_policy` function was built for the frozenlake-v0 env so it will break on any other env. You could simply ignore it or rewrite it for your env.
* in function `sample_reward`, __make sure t_max steps is enough to solve the environment__ even if agent is sometimes acting suboptimally. To estimate that, run several sessions without time limit and measure their length.

# Preparations
**Similar code as before:**

In [144]:
import gym
#from gym import wrappers

#create a single game instance
env = gym.make("Taxi-v2")
env = wrappers.Monitor(env, '/tmp/taxi-v2', force=True)

#start new game
env.reset();

[2017-03-13 03:03:18,713] Making new env: Taxi-v2
[2017-03-13 03:03:18,730] Clearing 24 monitor files from previous run (because force=True was provided)
[2017-03-13 03:03:19,098] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video000000.json


In [145]:
import numpy as np
n_states = env.observation_space.n
n_actions = env.action_space.n
def get_random_policy(pool=[]):
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    # randint(0, 4, 16) returns an array of integers between 0 and 3 inclusive
    # (or, in other words, starting from 0 to below 4)
    # and the third therm (16), will be the array size
    rand_pol = np.random.randint(0, n_actions, n_states)
    while rand_pol in pool:
        #it will loop until a NEW random policy appear
        print("rand_pol:", rand_pol)
        print("pool: ", pool)
        rand_pol = np.random.randint(0, n_actions, n_states)
        
    return rand_pol

In [146]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]
assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should match n_actions-1'
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas, [1. / n_actions] * n_actions, atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print("Seems fine!")

Action frequencies over 10^4 samples: [ 0.1668264  0.1667818  0.166578   0.166587   0.166508   0.1667188]
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [147]:
def sample_reward(env, policy, t_max=20000):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    #s: state; where our actor is
    s = env.reset()
    total_reward = 0
    t = 0
    is_done = False

    while t < t_max and not is_done:
        # p = policy: with probabilities equal to the ones returned by get_random_policy()
        s, reward, is_done, _ = env.step(policy[s])
        # accumulating rewards
        total_reward += reward
        t+=1
    #s = env.reset()
    return total_reward

In [148]:
print("generating 10^3 sessions...")
rewards = [sample_reward(env,get_random_policy()) for _ in range(10**3)]
assert all([type(r) in (int, float) for r in rewards]), 'sample_reward must return a single number'
#assert all([0 <= r <= 1 for r in rewards]), 'total rewards should be between 0 and 1 for frozenlake (if solving taxi, delete this line)'
print("Looks good!")

[2017-03-13 03:03:21,260] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video000001.json


generating 10^3 sessions...


[2017-03-13 03:03:21,348] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video000008.json
[2017-03-13 03:03:21,500] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video000027.json
[2017-03-13 03:03:21,756] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video000064.json
[2017-03-13 03:03:22,102] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video000125.json
[2017-03-13 03:03:22,589] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video000216.json
[2017-03-13 03:03:23,372] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video000343.json
[2017-03-13 03:03:24,380] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video000512.json
[2017-03-13 03:03:25,586] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video000729.json
[2017-03-13 03:03:27,182] Starting new video rec

Looks good!


In [149]:
def evaluate(policy, best_score=0, n_times=10):
    """Run several evaluations and average the score the policy gets."""
    # rewards: array with n_times (100) elements consisting of the total_rewards returned by sample_reward()
    if best_score > -200:
        rewards = [sample_reward(env, policy) for n in range(n_times*10)]
    else:
        rewards = [sample_reward(env, policy) for n in range(n_times)]
    return float(np.mean(rewards))

#### Ignoring the random search, jumping right into
# Part II - Genetic Algorithms

In [150]:
def print_policy(policy):
    """a function that displays a policy in a human-readable way."""
    """
    taxi =  R: | : :G
             : : : : 
             : : : : 
             | : | : 
            Y| : |B: 
    """
    taxi = "R: | : :G : : : :  : : : :  | : | : Y| : |B: "
    assert env.spec.id == "Taxi-v2", "this function only works with taxi-v2"

    
    # where to move from each tile (we're a bit unsure if this is accurate)
    actions = ['>^v<pd'[a] for a in policy]
    
    #draw arrows above S and F only
    signs = [action if tile in " RGYB" else tile for action, tile in zip(actions, taxi)]
    
    for i in range(0, 45, 9):
        print(' '.join(signs[i:i+9]))

print("random policy:")
print_policy(get_random_policy())

random policy:
< : d | d : v : p
p : < : p : d : p
v : p : d : p : p
< | < : > | < : ^
< | ^ : d | < : p


In [151]:
def crossover(policy1, policy2, p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    # policyx: [0,1,3,2,1,0,3,2,1,0,3,2,0,2,2,1]
    new_pol = []
    for i in range(len(policy1)):
        #choosing between the ith element between pol1 and pol2 with probability p
        new_pol.append(np.random.choice((policy1[i], policy2[i]), p=[p, 1-p]))
        
    return new_pol

In [152]:
def mutation(policy, p=0.01):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    # if we modify "policy" directly, we'll change the value of policy. Lists work that way, so we
    # need to use a copy
    mutated_policy = list(policy)
    #n_actions = env.action_space.n
    for a in policy:
        # with 1% probability, we mutate element a from policy
        if np.random.choice((0,1), p=[1-p, p]):
            mutated_policy[a] = np.random.randint(0, n_actions)
    return mutated_policy

In [153]:
np.random.seed(1234)
policies = [crossover(get_random_policy(), get_random_policy()) 
            for i in range(10**2)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should be n_actions-1'

assert any([np.mean(crossover(np.zeros(n_states), np.ones(n_states))) not in (0, 1)
               for _ in range(100)]), "Make sure your crossover changes each action independently"
print("Seems fine!")

Seems fine!


In [154]:
n_epochs = 10000 #default: 100 - how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 100 #how many crossovers to make on each step
n_mutations = 200 #how many mutations to make on each tick
n_low_scorers = 0 #how many random low-scorer policies, not into the best scorers, can pass to the next pool

In [155]:
print("initializing...")
#pool = <spawn a list of pool_size random policies>
pool = [get_random_policy() for i in range(pool_size)]
#pool_scores = <evaluate every policy in the pool, return list of scores>
pool_scores = [evaluate(p) for p in pool]

initializing...


[2017-03-13 03:03:36,898] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video002000.json
[2017-03-13 03:03:41,915] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video003000.json
[2017-03-13 03:03:46,867] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video004000.json
[2017-03-13 03:03:51,850] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video005000.json
[2017-03-13 03:03:56,837] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video006000.json
[2017-03-13 03:04:08,167] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video007000.json
[2017-03-13 03:04:13,153] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video008000.json
[2017-03-13 03:04:18,138] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video009000.json
[2017-03-13 03:04:23,372] Starting new video rec

In [156]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float, int) for score in pool_scores])

# Diverse Pool

### As with 4.-moar

In [157]:
#main loop
from tqdm import tqdm
#for epoch in range(n_epochs):
for epoch in tqdm(range(n_epochs)):
    print("Epoch %s:"%epoch)
    
    # 1. Removing duplicates from pool
    # converting list of np arrays to list of tuples because I couldn't find a better way to remove duplicates
    uniques_pool = [tuple(policy) for policy in pool]
    # set() returns only unique elements. We need to convert them to np arrays again
    uniques_pool = [np.asarray(policy) for policy in set(uniques_pool)]
    if len(uniques_pool) != len(pool):
        # We found some duplicates
        print("We found", len(pool) - len(uniques_pool), "duplicated policies at this epoch!")
        pool = uniques_pool
        # we should fill pool with new random policies to keep it size, but with the code below
        # it will be maintain it's size.
    # We could check for duplicates on crossovered or mutated,
    #but it's more similar code and I want to finish this one
    
    # evaluation policies before crossovering them:
    pool_scores = [evaluate(p) for p in pool]
    # we'll select the best n_crossovers (50) policies to mix between them
    # we could use another number of best policies instead of n_crossovers (50),
    # but it's late and I don't know what I'm doing
    selected_indices = np.argsort(pool_scores)[-n_crossovers:]
    
    #crossovered = <crossover random guys from pool, n_crossovers total>
    # using selected_indices as a fraction of pool with 50 best scores
    crossovered = [crossover(pool[np.random.choice(selected_indices)], 
                             pool[np.random.choice(len(pool))]) 
                   for c in range(n_crossovers)]
    # from now on it's all the same: mutations, adding all to a pool, evaluating (again) and selecting
    # best scores. Repeat for n_epochs.
    #mutated = <add several new policies at random, n_mutations total>
    mutated = [mutation(pool[np.random.choice(len(pool))]) 
               for m in range(n_mutations)]
    
    assert type(crossovered) == type(mutated) == list
    
    #add new policies to the pool
    #pool = <add up old population with crossovers/mutations>
    #plus sing (+) concatenates lists in python
    pool = pool + crossovered + mutated
    #pool_scores = <evaluate all policies again>
    pool_scores = [evaluate(p) for p in pool]
    
    # 2. Adding a couple of random low scorers to the final pool
    # select pool_size-n_low_scorers best policies. we'll add n_low_scorers later 
    selected_indices = np.argsort(pool_scores)[-pool_size+n_low_scorers:]
    # Now we need to add n_low_scorers to our indices
    # np.argsort(pool_scores)[:-pool_size+n_low_scorers] will contain all indices NOT used abobe
    # so we need to choose n_low_scorers random indices from there
    low_scorers_indices = np.random.choice(np.argsort(pool_scores)[:-pool_size+n_low_scorers], n_low_scorers)
    # now we need to concatenate all indices into one numpy array
    selected_indices = np.concatenate((selected_indices, low_scorers_indices))
    #filling pool only with best values
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    print("Best score:", pool_scores[-1])
    print_policy(pool[-1])
    print("")
env.close()


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch 0:



[2017-03-13 03:04:34,229] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video012000.json
[2017-03-13 03:04:39,842] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video013000.json
[2017-03-13 03:04:44,848] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video014000.json
[2017-03-13 03:04:50,131] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video015000.json
[2017-03-13 03:04:55,365] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video016000.json
[2017-03-13 03:05:00,365] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video017000.json
[2017-03-13 03:05:05,355] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video018000.json
[2017-03-13 03:05:10,314] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video019000.json
[2017-03-13 03:05:15,289] Starting new video re

Best score: -881.93
> : v | v : p : ^
d : ^ : ^ : p : ^
d : p : v : p : d
^ | v : > | < : <
> | ^ : > | ^ : <

Epoch 1:


[2017-03-13 03:08:50,855] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video062000.json
[2017-03-13 03:08:55,928] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video063000.json
[2017-03-13 03:09:01,013] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video064000.json
[2017-03-13 03:09:06,179] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video065000.json
[2017-03-13 03:09:11,234] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video066000.json
[2017-03-13 03:09:16,260] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video067000.json
[2017-03-13 03:09:21,396] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video068000.json
[2017-03-13 03:09:26,538] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video069000.json
[2017-03-13 03:09:31,757] Starting new video rec

...

Best score: -236.0
v : v | > : ^ : ^
d : ^ : > : d : <
v : d : < : > : d
v | ^ : p | ^ : <
d | > : v | ^ : <

Epoch 223:


[2017-03-13 18:57:05,775] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11143000.json
[2017-03-13 18:57:10,764] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11144000.json
[2017-03-13 18:57:15,959] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11145000.json
[2017-03-13 18:57:20,960] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11146000.json
[2017-03-13 18:57:26,057] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11147000.json
[2017-03-13 18:57:31,274] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11148000.json
[2017-03-13 18:57:36,359] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11149000.json
[2017-03-13 18:57:41,364] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11150000.json
[2017-03-13 18:57:46,578] Starti

Best score: -254.0
> : > | < : ^ : ^
d : ^ : > : d : <
v : d : < : > : d
v | ^ : p | ^ : <
d | > : v | ^ : <

Epoch 224:


[2017-03-13 19:01:22,376] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11193000.json
[2017-03-13 19:01:27,414] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11194000.json
[2017-03-13 19:01:32,437] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11195000.json
[2017-03-13 19:01:37,428] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11196000.json
[2017-03-13 19:01:42,424] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11197000.json
[2017-03-13 19:01:47,531] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11198000.json
[2017-03-13 19:01:52,540] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11199000.json
[2017-03-13 19:01:57,679] Starting new video recorder writing to /tmp/taxi-v2/openaigym.video.1.27766.video11200000.json
[2017-03-13 19:02:02,704] Starti

KeyboardInterrupt: 

In [ ]:
#from gym import wrappers
#env = gym.make('CartPole-v0')
#env = wrappers.Monitor(env, '/tmp/cartpole-experiment-1')
#for i_episode in range(20):
#    observation = env.reset()
#    for t in range(100):
#        env.render()
#        print(observation)
#        action = env.action_space.sample()
#        observation, reward, done, info = env.step(action)
#        if done:
#            print("Episode finished after {} timesteps".format(t+1))
#            break